In [212]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from copy import deepcopy
from EventDriven.backtest import OptionSignalBacktest, RiskManager
from pydantic.dataclasses import dataclass as pydantic_dataclass
from pydantic import Field, ConfigDict
from typing import List, Optional, Dict, Any, Union
import numbers
from datetime import datetime, date

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import json
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
_key = 11
with open(f'../input/profitable_weights_{_key}.json', 'r') as f:
    weights = json.load(f)
ttrades__ = pd.read_csv(f'../input/profitable_trades_{_key}.csv').iloc[:, 1:]
ttrades__['Duration'] = ttrades__.Duration.apply(lambda x: int(x.split(' ')[0]))
tick = ['AAPL', 'NFLX', 'NVDA']
# ttrades__ = ttrades__[(ttrades__.Ticker.isin(tick))]
# trades_ = ttrades__.iloc[0:10, :]
trades_ = ttrades__
# trades_ = ttrades__.iloc[9, :].to_frame().T
# trades_.loc[17, 'Size'] = -126
# ttrades__
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,24,502,511,249.189127,202.630005,-1117.418939,-0.186843,2024-01-03,2024-01-17,14,BA
1,66,502,533,184.864771,183.419998,-95.355022,-0.007815,2024-01-03,2024-02-16,44,AAPL
2,120,502,585,136.184992,145.509995,1119.000332,0.068473,2024-01-03,2024-05-02,120,AMD
3,45,502,637,346.187441,475.000000,5796.565154,0.372089,2024-01-03,2024-07-18,197,META
4,16,502,752,653.489257,923.650024,4322.572278,0.413413,2024-01-03,2024-12-31,363,COST
5,14,502,752,468.955627,901.799988,6059.821046,0.922996,2024-01-03,2024-12-31,363,NFLX
6,600,509,752,54.811169,138.029999,49931.297911,1.518282,2024-01-12,2024-12-31,354,NVDA
7,20,521,643,157.549500,180.389999,456.809988,0.144973,2024-01-31,2024-07-26,177,AMZN
8,90,529,530,97.259218,94.849998,-216.829772,-0.024771,2024-02-12,2024-02-13,1,SBUX
9,112,588,617,156.867127,156.990005,13.762352,0.000783,2024-05-07,2024-06-18,42,AMD


In [5]:
symbol_list = trades_.Ticker.unique()
untraded_symbols = [s for s in weights.keys() if s not in trades_.Ticker.unique()]
for s in untraded_symbols:
    weights.pop(s)


max_cash = {}
cash = 20_000
for s, w in weights.items():
    print(f'{s} {w} {w * cash}')
    if w * cash > 500:
        max_cash[s] = 4
    elif w * cash > 300:
        max_cash[s] = 3
    elif w * cash > 200:
        max_cash[s] = 2
    elif w * cash > 100:
        max_cash[s] = 1
    else:
        max_cash[s] = 0.5
max_cash
pd.Series(weights).sort_values(ascending=False), max_cash

NVDA 0.2527323831217408 5054.647662434816
TSLA 0.1315097599965802 2630.1951999316043
AMD 0.1264165402671991 2528.330805343982
META 0.12170653973488622 2434.1307946977245
AAPL 0.09471622942745511 1894.3245885491021
NFLX 0.05117009979641809 1023.4019959283618
COST 0.08262689789608499 1652.5379579216997
SBUX 0.06740991086070897 1348.1982172141793
BA 0.04636898185718145 927.379637143629
AMZN 0.025342657041745188 506.85314083490374


(NVDA    0.252732
 TSLA    0.131510
 AMD     0.126417
 META    0.121707
 AAPL    0.094716
 COST    0.082627
 SBUX    0.067410
 NFLX    0.051170
 BA      0.046369
 AMZN    0.025343
 dtype: float64,
 {'NVDA': 4,
  'TSLA': 4,
  'AMD': 4,
  'META': 4,
  'AAPL': 4,
  'NFLX': 4,
  'COST': 4,
  'SBUX': 4,
  'BA': 4,
  'AMZN': 4})

In [208]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo

pd.options.display.max_rows = 50
pd.options.display.max_columns = 50

evb_backtest = OptionSignalBacktest(trades_, initial_capital=cash)
evb_backtest.portfolio.initial_capital
w_map = {x: w  * 0.85 for x, w in weights.items()} ## 75% of the weights for each stock
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.risk_manager.sizing_lev = 4.5
evb_backtest.portfolio.max_contract_price_factor = 2
evb_backtest.portfolio.min_moneyness_threshold = 3
evb_backtest.portfolio.order_settings =  {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.1},
                {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.1} 
            ],
            'name': 'vertical_spread',
            'strategy': 'vertical',
            'target_dte': 270,
            'structure_direction': 'long',
            'spread_ticks': 1,
            'dte_tolerance': 60,
            'min_moneyness': 0.75,
            'max_moneyness': 1.25,
            'min_total_price': 0.5
        }


evb_backtest.portfolio.max_contract_price = max_cash
evb_backtest.executor.commission_rate = 0.65/100
evb_backtest.portfolio.min_moneyness_threshold = 5
evb_backtest.executor.max_slippage_pct = 0.075
evb_backtest.portfolio.roll_map = 90
evb_backtest.portfolio.moneyness_width_factor = .025
evb_backtest.portfolio.dte_reduction_factor = 30
evb_backtest.portfolio.min_acceptable_dte_threshold = 95
evb_backtest.portfolio.risk_manager.limits['dte'] = True
evb_backtest.portfolio.risk_manager.limits['delta'] = True
evb_backtest.portfolio.risk_manager.limits['moneyness'] = True
evb_backtest.portfolio.risk_manager.max_moneyness = 1.05
for key  in max_cash:
    if max_cash[key]*100 > evb_backtest.portfolio.allocated_cash_map[key]:
        print(key, max_cash[key]*100, evb_backtest.portfolio.allocated_cash_map[key])

evb_backtest.portfolio.risk_manager.print_settings()

signals = evb_backtest.bars.signal_df
signals_df = deepcopy(signals).set_index('Date')
((signals_df!=-1)&(signals_df!=-0)).sum().sum()
rm = evb_backtest.portfolio.risk_manager
pm = evb_backtest.portfolio

Event date 20240115 not found in backtest range.
2025-11-16 20:43:16 OptionSignalEventScheduler ERROR: Event date 20240115 not found in backtest range
Event date 20240219 not found in backtest range.
2025-11-16 20:43:16 OptionSignalEventScheduler ERROR: Event date 20240219 not found in backtest range
Event date 20240329 not found in backtest range.
2025-11-16 20:43:16 OptionSignalEventScheduler ERROR: Event date 20240329 not found in backtest range
Event date 20240527 not found in backtest range.
2025-11-16 20:43:16 OptionSignalEventScheduler ERROR: Event date 20240527 not found in backtest range
Event date 20240619 not found in backtest range.
2025-11-16 20:43:16 OptionSignalEventScheduler ERROR: Event date 20240619 not found in backtest range
Event date 20240704 not found in backtest range.
2025-11-16 20:43:16 OptionSignalEventScheduler ERROR: Event date 20240704 not found in backtest range
Event date 20240902 not found in backtest range.
2025-11-16 20:43:16 OptionSignalEventSchedule

In [183]:
## 

@pydantic_dataclass(config=ConfigDict(arbitrary_types_allowed=True))
class RiskManagerOrderRequest:
    """
    Dataclass representing an order request generated for risk manager to process.
    """
    symbol: str
    date: Union[datetime, str]
    signal_id: str
    max_close: numbers.Number
    direction: str

### Order Schema Streamline


In [215]:
from EventDriven.riskmanager.picker.order_picker import OrderPicker
from EventDriven.riskmanager._orders import get_open_order
from EventDriven.riskmanager._order_validator import OrderInputs
from EventDriven.riskmanager.picker import OrderSchema
from EventDriven.configs.core import OrderResolutionConfig
from EventDriven.dataclasses.orders import OrderRequest
from EventDriven.configs.base import BaseConfigs

In [203]:
BaseConfigs._registry

{<weakref at 0x161384770; to 'type' at 0x7fcd16485e10 (ChainConfig)>, <weakref at 0x161386930; to 'ABCMeta' at 0x7fcd16557ad0 (ZscoreSizerConfigs)>, <weakref at 0x161386700; to 'type' at 0x7fcd0a3f8340 (OrderSchemaConfigs)>, <weakref at 0x161386200; to 'ABCMeta' at 0x7fcd1711b540 (DefaultSizerConfigs)>, <weakref at 0x161385e90; to 'type' at 0x7fcd16610340 (OrderResolutionConfig)>, <weakref at 0x161384b30; to 'type' at 0x7fcd131875b0 (OrderPickerConfig)>}

In [205]:
# BaseConfigs.display_and_describe_all_configs()
OrderResolutionConfig().display_and_describe_configs()


Current Configuration Settings for OrderResolutionConfig:
{'resolve_enabled': True, 'otm_moneyness_width': 0.1, 'itm_moneyness_width': 0.1, 'max_close': 10.0, 'max_tries': 6, 'max_dte_tolerance': 120}
        

Configuration Descriptions for OrderResolutionConfig:
- resolve_enabled: True  # Flag to enable or disable order resolution.
- otm_moneyness_width: 0.1  # This should be a float representing the OTM moneyness width max for ATM against OTM.
- itm_moneyness_width: 0.1  # This should be a float representing the ITM moneyness width max for ATM against ITM.
- max_close: 10.0  # Maximum close price allowed for the order.
- max_tries: 6  # Maximum number of attempts to resolve an order schema.
- max_dte_tolerance: 120  # Minimum days to expiration tolerance allowed for the order.



In [ ]:

picker = OrderPicker(start_date="2025-01-01", end_date="2025-10-11")
picker.get_order_schema(ticker="SPY", option_type="P")
req = OrderRequest(
    date='2025-05-15',
    symbol='AAPL',
    option_type='P',
    max_close=5,
    spot=380,
    chain_spot=382,
    tick_cash=100,
)

# def get_order(self: OrderPicker, 
#               request: OrderRequest):
    
#     order_schema = self.get_order_schema(ticker=request.symbol, option_type=request.option_type, max_total_price=request.max_close)
#     schema_as_tuple = tuple(order_schema.data.items())
#     order = self._get_order(
#         schema=schema_as_tuple,
#         date=request.date,
#         spot=request.spot,
#         chain_spot=request.chain_spot,
#         print_url=False
#     )
#     return order

# order = get_order(picker, req)
# order
req

OrderPickerOrderRequest(date='2025-05-15', symbol='AAPL', option_type='P', max_close=5, tick_cash=100, spot=380, chain_spot=382)

In [206]:
order_from_picker = picker.get_order(
    request=req
)
order_from_picker

HI BITCH
2025-11-16 20:41:20 EventDriven.riskmanager._order_validator WARNING: Unknown input key: resolve_enabled. This key will be ignored.
2025-11-16 20:41:22 QuantTools.EventDriven.riskmanager ERROR: Retrieved chain for AAPL on 2025-05-15


Order(signal_id=N/A), data=OrderData(trade_id=&L:AAPL20260116P255&S:AAPL20260116P250, quantity=1))

In [ ]:
def construct_inputs(self, 
                     request: OrderRequest,
                     schema: OrderSchema,
                     order_resolution_config: OrderResolutionConfig) -> OrderInputs:
    """
    Construct OrderInputs dataclass from OrderRequest and OrderSchema.
    """
    inputs = OrderInputs(
        tick = request.symbol,
        date = request.date,
        option_type = request.option_type,
        signal_id = "N/A",
        spot = request.spot,
        option_strategy = schema["strategy"],
        structure_direction = schema["structure_direction"],
        spread_ticks = schema.data.get("spread_ticks", 0),
        dte_tolerance = schema.data.get("dte_tolerance", 0),
        min_moneyness = schema.data.get("min_moneyness", 0),
        max_moneyness = schema.data.get("max_moneyness", float('inf')),
        target_dte = schema.data.get("target_dte", 0),
        min_total_price = schema.data.get("min_total_price", 0),
        direction = "N/A",
        **order_resolution_config.__dict__,

    )
    return inputs

construct_inputs(picker, req, schema, OrderResolutionConfig())


2025-11-16 19:38:34 EventDriven.riskmanager._order_validator WARNING: Unknown input key: resolve_enabled. This key will be ignored.


## RiskManager Streamline

### Breakout Get_order

In [219]:
from EventDriven.riskmanager.market_data import MarketTimeseries
from EventDriven.riskmanager.base import RiskManager
class Bar:
    def __init__(self):
        self.signal_df = pd.DataFrame()
        self.symbol_list = []
mt = MarketTimeseries()
rm = RiskManager(Bar(), None, None, None, None, None, None)

In [ ]:
def get_order(self: RiskManager,
              req: OrderRequest):
    
    """"""
    ## Generate Data
    self.generate_data(tick)

    ## Get order 
    order = self.order_picker.get_order(request=req)

SyntaxError: incomplete input (1693818882.py, line 7)